# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [114]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [115]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [116]:
def build_classifiers():
    classifiers_list = []
    lr = LinearRegression()
    knc = KNeighborsClassifier()
    svc = SVC()
    dtc = DecisionTreeClassifier()
    gnb = GaussianNB()
    qda = QuadraticDiscriminantAnalysis()
    lr.fit(data_set, labels)
    knc.fit(data_set, labels)
    svc.fit(data_set, labels)
    dtc.fit(data_set, labels)
    gnb.fit(data_set, labels)
    qda.fit(data_set, labels)
    classifiers_list.append(lr.fit(data_set, labels))
    classifiers_list.append(knc.fit(data_set, labels))
    classifiers_list.append(svc.fit(data_set, labels))
    classifiers_list.append(dtc.fit(data_set, labels))
    classifiers_list.append(gnb.fit(data_set, labels))
    classifiers_list.append(qda.fit(data_set, labels))

    return lr, knc, svc, dtc, gnb, qda # and here
type(build_classifiers())

tuple

In [125]:
#taking all possible 3-element sets of classifiers from a set of 6 classifiers
from itertools import *
my_combinations = list(combinations(build_classifiers(),3))

In [128]:
my_comb_names = list(combinations(['lr', 'knc', 'svc', 'dtc', 'gnb', 'qda'], 3))

In [99]:
#I added a variable st_class which is the classifier I am using as a stacked classifier
def build_stacked_classifier(classifiers,st_class):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    #output = np.array(output).reshape((130,2))
    
    # stacked classifier part:
    stacked_classifier = st_class
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    #stacked_classifier.fit(output.reshape((130,2)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [131]:
knc = KNeighborsClassifier()
svc = SVC()
dtc = DecisionTreeClassifier()
gnb = GaussianNB()
qda = QuadraticDiscriminantAnalysis()
tab_all = []
for c in [svc, dtc, qda]:
    tab_mid = []
    for comb in my_combinations:
        predicted = build_stacked_classifier(comb,c)
        accuracy = accuracy_score(test_labels, predicted)
        tab_mid.append(accuracy)
    tab_all.append(tab_mid)

/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/magdalena/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.

In [133]:
#final table: accuracy scores for all combinations of classifiers, 
#with svc, dtc or qda used as a stacked classifier.
import pandas as pd
df = pd.DataFrame(tab_all).T
df.columns = [ 'svc', 'dtc', 'qda']
df.index = my_comb_names
df

,svc,dtc,qda
"(lr, knc, svc)",0.00,0.10,0.30
"(lr, knc, dtc)",0.75,0.70,0.65
"(lr, knc, gnb)",0.70,0.70,0.95
"(lr, knc, qda)",0.75,0.75,0.80
"(lr, svc, dtc)",0.60,0.60,0.55
"(lr, svc, gnb)",0.60,0.60,0.90
"(lr, svc, qda)",0.65,0.60,0.70
"(lr, dtc, gnb)",0.00,0.15,0.40
"(lr, dtc, qda)",0.65,0.75,0.20
"(lr, gnb, qda)",0.65,0.60,0.65


In [137]:
df.max()

svc    0.90
dtc    0.85
qda    0.95
dtype: float64

In [138]:
df["qda"].idxmax() #best combination of classifiers, with stacked classifier qda

('lr', 'knc', 'gnb')

In [140]:
df["qda"].max() #best combination of classifiers accuracy

0.95

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [185]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0


def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

Fill the two functions below:

Calculate the new weights $w_{n}^{(t+1)}=\frac{w_{n}^{(t)}\exp{\alpha_{t}I(y_{n}\neq h_{t}(x_{n}))}}{Z_{t}}$:


* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,

In [186]:
def set_new_weights(model): 
    new_weights = calculate_accuracy_vector(model.predict(test_set), test_labels)
    for i in new_weights:
        i = i+1.0
    Zt = np.sum(new_weights)
    return new_weights / Zt


Train the classifier with the code below:

In [187]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.         0.00195695 0.00195695 0.         0.00195695 0.00195695
 0.00195695 0.         0.00195695 0.         0.00195695 0.00195695
 0.00195695 0.         0.00195695 0.00195695 0.         0.00195695
 0.         0.00195695 0.00195695 0.         0.         0.
 0.         0.         0.00195695 0.00195695 0.         0.00195695
 0.         0.         0.         0.         0.00195695 0.00195695
 0.00195695 0.         0.00195695 0.00195695 0.         0.
 0.00195695 0.00195695 0.         0.00195695 0.00195695 0.00195695
 0.         0.00195695 0.         0.00195695 0.         0.00195695
 0.         0.00195695 0.00195695 0.00195695 0.         0.00195695
 0.         0.         0.00195695 0.         0.00195695 0.
 0.         0.00195695 0.00195695 0.         0.00195695 0.00195695
 0.         0.         0.         0.         0.00195695 0.00195695
 0.         0.00195695 0.         0.         0.         0.
 0.00195695 0.         0.00195695 0.00195695 0.00195695 0.00195695
 0.         0.00195695 0.  

Set the validation data set:

In [194]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [199]:
def get_prediction(x):
    predictions = []
    for i in range(len(classifiers)):
        predicted = classifiers[i].predict(x)
        predictions.append(predicted)
    scores = [0.0, 0.0]  
    for prediction, idx_clf in enumerate(predictions):
        if prediction == 0:
            scores[0] = scores[0] + weights[idx_clf]
        else:
            scores[1] = scores[1] + weights[idx_clf]
    if scores[0] >= scores[1]:
        return 0
    else:
        return 1

Test it:

In [200]:
prediction = get_prediction(validate_x)

print(prediction)

0
